# Level 2 - Week 1 - 01 Requirements to Architecture

**Estimated time:** 60-90 minutes

## Learning Objectives

- Write user journeys and failure journeys
- Map journeys to system components
- List debug artifacts like request_id


## Overview

Translate requirements into clear journeys before you design endpoints.
Focus on happy path, admin path, failure path, and observability.

## Practice Steps

- Write journeys as short bullet stories with example questions.
- Map journeys to components and data flow.


### Sample code

Example journey map with components and debug fields.


In [ ]:
journeys = {
    'happy_path': ['user asks question', 'retrieve context', 'answer with citations'],
    'admin_path': ['admin ingests doc', 'index updated'],
    'failure_path': ['retrieval empty', 'return clarify'],
    'observability': ['request_id logged', 'retrieval hits logged', 'mode logged'],
}

components = ['api', 'retrieval', 'generation', 'storage']
print(journeys)
print(components)


### Student fill-in

Add your own journeys and map them to components.


In [ ]:
REQUIREMENTS = {
    'happy_path': [],
    'admin_path': [],
    'failure_path': [],
    'observability': [],
}

COMPONENTS = {
    'api': '',
    'retrieval': '',
    'generation': '',
    'storage': '',
}

# TODO: fill in REQUIREMENTS and COMPONENTS


## Self-check

- Do you have at least one failure journey?
- Did you include request_id or trace fields?


## Underlying theory: requirements → journeys → components → contracts

A requirement is not “an endpoint”. It is a constraint on system behavior.

A practical workflow:

1. **Journeys**: write short stories for:
   - user journey (happy path)
   - admin journey (ingestion / config)
   - failure journey (no evidence, provider down)
   - observability journey (how you debug)

2. **Components**: map each journey to responsibilities:
   - API layer (contracts + auth + budgets)
   - retrieval (search + filters)
   - generation (prompting + modes)
   - validation (citations, schema)
   - storage (vector DB + metadata)

3. **Contracts**: each boundary needs inputs/outputs and failure modes:
   - request/response schemas
   - status codes / error payload shape
   - invariants (e.g. “answer mode must be one of {answer, clarify, refuse}”)

4. **Debug artifacts** (non-negotiable):
   - `request_id`
   - stage-level logs (retrieve vs generate vs validate)
   - retrieved chunk ids + scores for any answer

## Practice: Requirements → Components → Contracts

In Level 2 you formalize **system thinking**: requirements → components → contracts.

You will:

1. Translate requirements into a system diagram (clients, API, storage, background jobs).
2. Define request/response contracts.
3. Sketch a minimal FastAPI skeleton with clear boundaries and error handling.

### Practice Steps

- Draft 3–5 requirements as bullet points.
- Map them to components and data flow.
- Fill in the API models + endpoints below.
- Add logging + error handling hooks.

Constraint:

- keep contracts small and explicit (don’t hide behavior inside “free-form text”)

In [ ]:
TASK_1_1_TEMPLATE = """Task: Requirements → components

1) Write 3–5 requirements for a simple AI helpdesk.

Requirements (examples):
- Users can ask questions about policies.
- Admins can ingest policy docs.
- Answers must include citations when answering.
- If retrieval is empty, the system must clarify/refuse.

Fill in:

REQUIREMENTS:
1.
2.
3.
4.
5.

2) Map requirements to components.

COMPONENTS AND RESPONSIBILITIES:
- API layer:
- Retrieval/search:
- Context assembly + grounding:
- LLM generation:
- Validation (citations/mode/schema):
- Storage (vector DB + metadata):
- Observability (request_id, logs, traces):

3) For each component, list one observable artifact you would log.

OBSERVABILITY ARTIFACTS:
- API:
- Retrieval:
- Generation:
- Validation:
"""

print(TASK_1_1_TEMPLATE)

### Task: Define API contracts

Define request/response models for `/search` and `/ingest`.

Rules:

- keep models small and explicit
- encode defaults in the schema (e.g. `top_k=5`)
- plan for failure modes (e.g. validation errors, auth errors)

Goal:

- you should be able to look at the models and understand what the API guarantees

In [ ]:
from pydantic import BaseModel
from typing import Dict, List, Optional

# TODO: fill in request/response models
class SearchRequest(BaseModel):
    query: str
    top_k: int = 5
    filters: Optional[Dict] = None


class SearchHit(BaseModel):
    chunk_id: str
    doc_id: str
    score: float
    text: str
    metadata: Dict


class SearchResponse(BaseModel):
    hits: List[SearchHit]


class IngestRequest(BaseModel):
    doc_id: str
    text: str
    metadata: Dict | None = None


class IngestResponse(BaseModel):
    status: str
    chunks_indexed: int


### Task: FastAPI skeleton

Create a minimal FastAPI app with:

- `GET /health`
- `POST /search`
- `POST /ingest`

Requirements:

- include structured logging hooks (at minimum log `request_id` + `path` + `status_code`)
- treat `/search` as a debug-friendly endpoint (even if `/chat` is your main endpoint later)

Goal:

- the skeleton should be runnable and provide a clear place to plug in real retrieval/ingestion later

In [ ]:
from fastapi import FastAPI
import logging

app = FastAPI(title="Level 2 Service")
logger = logging.getLogger("level2")


@app.get("/health")
def health():
    return {"status": "ok"}


@app.post("/search", response_model=SearchResponse)
def search(req: SearchRequest):
    # TODO: replace with real retrieval
    logger.info("search request", extra={"query": req.query})
    return SearchResponse(hits=[])


@app.post("/ingest", response_model=IngestResponse)
def ingest(req: IngestRequest):
    # TODO: replace with real ingestion
    logger.info("ingest request", extra={"doc_id": req.doc_id})
    return IngestResponse(status="ok", chunks_indexed=0)
